In [1]:
# General
import os
import pandas as pd
import numpy as np
#from tqdm.notebook import tqdm
from tqdm import tqdm
import cv2
import warnings; warnings.simplefilter("ignore", UserWarning)

# Metrics
from sklearn.metrics import roc_auc_score

# Machine Learning Utilities
from sklearn.model_selection import train_test_split 

# Deep Learning
import torch
#import torchvision
import timm
from timm import create_model

import torch.nn as nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
#from torch.optim.lr_scheduler import CosineAnnealingLR
from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer

# Image Augmentation
#import albumentations
#from albumentations.pytorch.transforms import ToTensorV2

In [2]:
# Device Type
# Device Optimization
if torch.cuda.is_available():
    device = 'gpu'  # torch.device('cuda')
else:
    device = 'cpu'  # torch.device('cpu')    
#print(f'Using device: {device}')
print(f'Using device: '+device)

Using device: cpu


Now, lets set a fixed random seed for repeatability.

In [3]:
set_seed(999,reproducible=True)

# Training Data
Now, lets load and inspect the training data's metadata CSV file.

In [4]:
df = pd.read_csv(os.path.join('labels','train_labels.csv'))
df.head(3)

,id,target
0,00034abb3629,0
1,0004300a0b9b,0
2,000453852fda,0


Lets now add a column to detail the file path to the actual data file for each entry for the training set metadata and inspect the outcome.

In [5]:
train_data_dir = os.path.join('/', 'data', 'train')
df['path'] = df['id'].apply(lambda x: train_data_dir+os.path.join('/')+f'{x[0]}/{x}.npy')
df.head(3)

,id,target,path
0,00034abb3629,0,/data/train/0/00034abb3629.npy
1,0004300a0b9b,0,/data/train/0/0004300a0b9b.npy
2,000453852fda,0,/data/train/0/000453852fda.npy


Lets now split the data frame with training data into subsets for training an validation. We'll use a fixed seed to do so in a repeatable manner.

In [6]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=999)

## Generate a Dataset for Pytorch
Lets now create an application-specific data class. The data class below enabled both channelized and spatial approaches along with the options to use only the three 'ON' channels or all six channels, including the 'OFF' channels. 

TODO: The data class may also be improved with some data augmentation routines to increase the datapoints available for training. 

For this notebook, we will train with the spatial approach that leverages all six channels.

In [7]:
class SETIDataset:
    def __init__(self, df, spatial=True, sixchan=True):
        self.df = df
        self.spatial = spatial  # Whether to use a spatial or channelized orrientation
        self.sixchan = sixchan  # Whether to use all six channels or just the three 'ON' channels
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        label = self.df.iloc[index].target
        filename = self.df.iloc[index].path
        data = np.load(filename).astype(np.float32)
        if not self.sixchan: data = data[::2].astype(np.float32)
        if self.spatial:
            data = np.vstack(data).transpose((1, 0))
            data = cv2.resize(data, dsize=(256,256))     
            data_tensor = torch.tensor(data).float().unsqueeze(0)
        else:
            data = np.transpose(data, (1,2,0))
            data = cv2.resize(data, dsize=(256,256))     
            data = np.transpose(data, (2, 0, 1)).astype(np.float32)
            data_tensor = torch.tensor(data).float()

        return (data_tensor, torch.tensor(label))

Lets take our previously created train and validation dataframes and use the `SETIDataset` class to generate data sets suitable for ingestion in a Pytorch model.

In [8]:
train_ds = SETIDataset(train_df)
valid_ds = SETIDataset(valid_df)

In [9]:
bs = 128 # batch size - range from 8 to 128+ depending on available VRAM on the GPU
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=8)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs, num_workers=8)

In [10]:
dls = DataLoaders(train_dl, valid_dl)

# Training
We will use Zachary Mueller's `timm_learner` function to create an already-instantiated `Learner` object with the `DataLoaders` and an appropriately defined CNN model taken from Ross Wightman's amazing `timm` package. The code for `timm_learner` (see the hidden cell below) is based on fastai's `cnn_learner` function. We can tell `timm_learner` what CNN backbone we want to use, as well as the number of input and output channels, and fastai automatically defines the appropriate model. We also pass in the metrics and the loss function. Fastai's default optimizer is AdamW. Finally, we can also use mixed precision training easily.

We'll use a simple ImageNet-pretrained ResNext50_32x4d model.

In [11]:
def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_in=3, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, n_in=n_in, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

In [12]:
def roc_auc(preds,targ):
    try: return roc_auc_score(targ.cpu(),preds.squeeze().cpu())
    except: return 0.5

In [13]:
learn = timm_learner(dls,'resnext50_32x4d',pretrained=True,n_in=1,n_out=1,metrics=[roc_auc], opt_func=ranger, loss_func=BCEWithLogitsLossFlat()).to_fp16()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50_32x4d_ra-d733960d.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


Next, lets employ a fastai function to solve for an optimal learning rate.

In [ ]:
learn.lr_find(show_plot=True)

The idea is that the learning rate where the loss decreases the most is likely the best learning rate. In this case, this is around ~3e-2.

Let's fine-tune the pretrained model using fastai's fit_one_cycle function to train the frozen pretrained model with a one-cycle learning rate schedule. I use high weight decay regularization to prevent overfitting.

In [ ]:
learn.fit_one_cycle(10, 0.1, cbs=[ReduceLROnPlateau()])

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn = learn.to_fp32()

Now lets save our model for use later.

In [ ]:
learn.save(os.path.join('models', 'resnext50_32x4d-10epochs-128batch'))
learn = learn.load(os.path.join('models', 'resnext50_32x4d-10epochs-128batch'))

# Inference
Begin with the metadata file for the test data set and append file paths using the same methods applied to the training set.

In [ ]:
test_df = pd.read_csv(os.path.join(os.path.join('labels','sample_submission.csv'))

test_dir = os.path.join('/', 'data', 'test')
test_df['path'] = test_df['id'].apply(lambda x: test_dir+os.path.join('/')+f'{x[0]}/{x}.npy')
test_df.head(3)

In [ ]:
test_ds = SETIDataset(test_df)

In [ ]:
bs = 128 # batch size - range from 8 to 128+ depending on available VRAM on the GPU
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs, num_workers=8, shuffle=False)

While fastai provides inference functions if we use their specific data API, in this case we used plain PyTorch dataloaders. So we'll just have to iterate over the dataloader and apply the model.

In [ ]:
preds = []
for xb, _ in tqdm(test_dl):
    if device == 'gpu':
        with torch.no_grad(): output = learn.model(xb.cuda())
        preds.append(torch.sigmoid(output.float()).squeeze().cpu())
    elif device == 'cpu':
        with torch.no_grad(): output = learn.model(xb.cpu())
        preds.append(torch.sigmoid(output.float()).squeeze().cpu())
    else:
        print('device not recognized')
preds = torch.cat(preds)    

Finally, generate the CSV file detailing the predicted target class for each entry in the Test data set for submission to Kaggle.

In [ ]:
sample_df = pd.read_csv(os.path.join('labels','sample_submission.csv'))
sample_df['target'] = preds
sample_df.to_csv(os.path.join('labels','resnext50_32x4d-10epochs-128batch.csv'), index=False)